In [8]:
#%%
import pandas as pd
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.corpus import stopwords
import sys
import subprocess
import json

In [ ]:
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))

factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


###preprocesing

In [ ]:
#preprocessing
df = pd.read_excel("artikel-cleaning-final/data_gabungan_normalized.xlsx")  

# Hapus duplikat
df = df.drop_duplicates(subset="title")

# Hapus baris yang tidak ada content
df = df.dropna(subset=["content"])

# Hapus kolom tag
if "tag" in df.columns:
    df = df.drop(columns=["tag"])

#hapus kolom description
if "description" in df.columns:
    df = df.drop(columns=["description"])

In [ ]:
# Preprocessing teks
def clean_text(text):
    # Case folding
    text = text.lower()

    # Hapus URL/link
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Hapus tanda baca
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Hapus karakter non-ascii (noise)
    text = re.sub(r"[^\x00-\x7f]", " ", text)

    return text

def remove_stopwords(text):
    tokens = text.split()
    filtered = [word for word in tokens if word not in stop_words]
    return " ".join(filtered)

def stemming(text):
    return stemmer.stem(text)

In [ ]:
for col in ["title", "content"]:
    df[col] = df[col].astype(str)
    df[col] = df[col].apply(clean_text)
    df[col] = df[col].apply(remove_stopwords)
    df[col] = df[col].apply(stemming)

with open("json-file/docs.jsonl", "w", encoding="utf-8") as f:
    for i, row in df.iterrows():
        doc = {
            "id": str(i+1),  
            "contents": f"{row['title']} {row['content']}",  
            "title": row["title"],       
            "link": row["link"],         
            "date": str(row["date"])     
        }
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")

In [3]:
import pandas as pd
from IPython.display import display

XLSX_FILE = "artikel-cleaning-final/data_gabungan_normalized.xlsx"  
JSONL_FILE = "json-file/docs.jsonl"   

print("  DATA ASLI SEBELUM PREPROCESSING ")

try:
    df_before = pd.read_excel(XLSX_FILE)
    display(df_before.head())

except FileNotFoundError:
    print(f"Peringatan: File tidak ditemukan di '{XLSX_FILE}'. Mohon periksa kembali path dan nama file Anda.")
except Exception as e:
    print(f"Terjadi error saat membaca file Excel: {e}")

print(" DATA SETELAH PREPROCESSING (dari JSONL)")

try:
    df_after = pd.read_json(JSONL_FILE, lines=True)
    display(df_after.head())

except FileNotFoundError:
    print(f"Peringatan: File tidak ditemukan di '{JSONL_FILE}'. Mohon periksa kembali path file Anda.")
except Exception as e:
    print(f"Terjadi error saat membaca file JSONL: {e}")

  DATA ASLI SEBELUM PREPROCESSING 


,title,Tag,link,date,description,content
0,20 Contoh Prompt Gemini AI Foto Studio Bareng ...,NaN,https://tekno.kompas.com/read/2025/09/21/12350...,"Minggu, 21 September 2025",NaN,KOMPAS.com - Pengguna mungkin tertarik untuk m...
1,2 Cara Cek Kuota XL Terbaru dengan Mudah dan P...,NaN,https://tekno.kompas.com/read/2025/09/21/11350...,"Minggu, 21 September 2025",NaN,KOMPAS.com - Cara cek kuota XL merupakan salah...
2,Honor Ungkap 3 Alasan Orang Indonesia Memilih ...,NaN,https://tekno.kompas.com/read/2025/09/21/11020...,"Minggu, 21 September 2025",NaN,KOMPAS.com – Pasar tablet di Indonesia kian ra...
3,"Jadwal MPL S16 Hari Ini, Minggu 21 September: ...",NaN,https://tekno.kompas.com/read/2025/09/21/10320...,"Minggu, 21 September 2025",NaN,KOMPAS.com — Turnamen MPL Indonesia Season 16 ...
4,"Terbang Dekat ""Air Force One"", Pilot Spirit Ai...",NaN,https://tekno.kompas.com/read/2025/09/21/10030...,"Minggu, 21 September 2025",NaN,KOMPAS.com – Sebuah insiden udara yang melibat...


 DATA SETELAH PREPROCESSING (dari JSONL)


,id,contents,title,link,date
0,1,20 contoh prompt gemini ai foto studio bareng ...,20 contoh prompt gemini ai foto studio bareng ...,https://tekno.kompas.com/read/2025/09/21/12350...,"Minggu, 21 September 2025"
1,2,2 cek kuota xl baru mudah praktis kompascom ce...,2 cek kuota xl baru mudah praktis,https://tekno.kompas.com/read/2025/09/21/11350...,"Minggu, 21 September 2025"
2,3,honor 3 alas orang indonesia pilih beli tablet...,honor 3 alas orang indonesia pilih beli tablet,https://tekno.kompas.com/read/2025/09/21/11020...,"Minggu, 21 September 2025"
3,4,jadwal mpl s16 minggu 21 september geek fam vs...,jadwal mpl s16 minggu 21 september geek fam vs...,https://tekno.kompas.com/read/2025/09/21/10320...,"Minggu, 21 September 2025"
4,5,terbang air force one pilot spirit airlines te...,terbang air force one pilot spirit airlines te...,https://tekno.kompas.com/read/2025/09/21/10030...,"Minggu, 21 September 2025"


In [13]:
cmd = [
    sys.executable, "-m", "pyserini.index",
    "--collection", "JsonCollection",
    "--input", "json-file",
    "--index", "my_index",
    "--generator", "DefaultLuceneDocumentGenerator",
    "--threads", "1",
    "--storePositions",
    "--storeDocvectors",
    "--storeRaw"
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

pyserini.index is deprecated, please use pyserini.index.lucene.
2025-09-23 20:01:12,646 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-09-23 20:01:12,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-09-23 20:01:12,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-09-23 20:01:12,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: json-file
2025-09-23 20:01:12,649 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-09-23 20:01:12,650 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: my_index
2025-09-23 20:01:12,651 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 1
2025-09-23 20:01:12,651 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + Optimize (me